In [ ]:
import time
import json
import datetime
import globus_sdk

from globus_sdk import TimerJob
from globus_compute_sdk import Executor
from globus_sdk.experimental.globus_app import UserApp

from globus_sdk.utils import slash_join

In [ ]:
CLIENT_ID = "c781864e-a9c9-482e-8db8-d58ac5962a86"
my_app = UserApp("crocus-user-app", client_id=CLIENT_ID)

flows_client = globus_sdk.FlowsClient(app=my_app)

In [ ]:
compute_endpoint = "a8ef913f-dea0-486b-b2a9-4220a9549274"

In [ ]:
wxt_function = "166e572b-bbd0-4590-bb71-abad9503426a"

In [ ]:
gce = Executor(endpoint_id=compute_endpoint)

In [ ]:
# Prepare payload for ESGF ingest-wxt
wxt_data = {
    "ndays": 1,
    "y": 2024,
    "m": 8,
    "d": 2,
    "site": 'NU',
    "hours": 1,
    "odir": "/Users/mgrover/git_repos/esgf-crocus-globus-flows"
}

# Start the task
future = gce.submit_to_registered_function(wxt_function, kwargs=wxt_data)

In [ ]:
# Wait and print the result
result = future.result()
print(result)

In [ ]:
flow_definition = {
    "Comment": "A CROCUS WXT flow",
    "StartAt": "TransferInput",
    "States": {
        "TransferInput": {
            "Comment": "Transfer input data",
            "Type": "Action",
            "ActionUrl": "https://transfer.actions.globus.org/transfer",
            "Parameters": {
                "source_endpoint.$": "$.input.source.id",
                "destination_endpoint.$": "$.input.destination.id",
                "DATA": [
                    {
                        "source_path.$": "$.input.source.path",
                        "destination_path.$": "$.input.destination.path",
                        "recursive": True,
                    }
                ]
            },
            "ResultPath": "$.TransferFiles",
            "WaitTime": 300,
            "Next": "ProcessWXT"
        },
        "ProcessWXT": {
            "Comment": "Collect WXT data from Sage",
            "Type": "Action",
            "ActionUrl": "https://compute.actions.globus.org/",
            "Parameters": {
                "endpoint.$": "$.input.compute_endpoint",
                "function.$": "$.input.wxt_function",
                "kwargs.$": "$.input.wxt_kwargs"
            },
            "ResultPath": "$.CROCUS_output",
            "WaitTime": 600,
            "End": True
        },
    }
}

In [ ]:
flow = flows_client.create_flow(title="CROCUS Flow", definition=flow_definition, input_schema={})
# flow = flows_client.update_flow(flow_id=flow_id, title="CROCUS Flow", definition=flow_definition, input_schema={})

In [ ]:
flow_input = {
    "input": {
        "source": {
            "id": "03e6a23b-fb93-11ef-985b-0207be7ee3a1",
            "path": "/Users/mgrover/git_repos/esgf-crocus-globus-flows/flows"
        },
        "destination": {
            "id": "03e6a23b-fb93-11ef-985b-0207be7ee3a1",
            "path": "/Users/mgrover/git_repos/esgf-crocus-globus-flows/"
        },
        "compute_endpoint": compute_endpoint,
        "wxt_kwargs": wxt_data,
        "wxt_function": wxt_function,
    }
}

In [ ]:
flow_id = flow['id']
flow_id

In [ ]:
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    app=my_app,
)

In [ ]:
run = specific_flow_client.run_flow(
  body=flow_input,
  label="CROCUS Example",
  tags=['CROCUS', 'example']
)

In [ ]:
# Get run details
# run = flows_client.get_run(run_id)

run_id = run['run_id']
run_status = run['status']
print("This flow can be monitored in the Web App:")
print(f"https://app.globus.org/runs/{run_id}")
print(f"Flow run started with ID: {run_id} - Status: {run_status}")

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')
    
# Run completed
print(json.dumps(run.data, indent=2))

# Add an input schema

This gives us the "Start" button on the flows GUI.

In [ ]:
input_schema = {
    "required": [
        "input"
    ],
    "properties": {
        "input": {
            "type": "object",
            "required": [
                "source",
                "destination",
                "compute_endpoint",
                "wxt_function",
                "wxt_kwargs"
            ],
            "properties": {
                "source": {
                    "type": "object",
                    "title": "Select source collection and path",
                    "description": "The source collection and path (path MUST end with a slash)",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "destination": {
                    "type": "object",
                    "title": "Select destination collection and path",
                    "description": "The destination collection and path (path MUST end with a slash); default collection is 'Globus Tutorials on ALCF Eagle'",
                    "format": "globus-collection",
                    "required": [
                        "id",
                        "path"
                    ],
                    "properties": {
                        "id": {
                            "type": "string",
                            "format": "uuid"
                        },
                        "path": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
                },
                "compute_endpoint": {
                    "type": "string",
                    "format": "uuid",
                    "default": compute_endpoint,
                    "title": "Globus Compute Endpoint ID",
                    "description": "The UUID of the Globus Compute endpoint where the function will run"
                },
                "wxt_function": {
                    "type": "string",
                    "format": "uuid",
                    "default": wxt_function,
                    "title": "Globus Compute Function ID",
                    "description": "The UUID of the function to invoke; must be registered with the Globus Compute service"
                },
                "wxt_kwargs": {
                    "type": "object",
                    "title": "Function Inputs",
                    "description": "Inputs to pass to the function",
                    "properties":  {
                        "ndays": {
                            "type": "integer",
                            "default": 1
                        },
                        "y": {
                            "type": "integer",
                            "default": 2024
                        },
                        "m": {
                            "type": "integer",
                            "default": 8
                        },
                        "d": {
                            "type": "integer",
                            "default": 1
                        },
                        "site": {
                            "type": "string",
                            "default": "NU"
                        },
                        "hours": {
                            "type": "integer",
                            "default": 1
                        },
                        "odir": {
                            "type": "string",
                            "default": "/home/rchard/src/CROCUS/output/"
                        },
                    },
                    "additionalProperties": False
                }
            },
            "additionalProperties": False
        }
    },
    "additionalProperties": False
}

In [ ]:
flow = flows_client.update_flow(flow_id=flow_id, title="CROCUS Flow", definition=flow_definition, input_schema=input_schema)

# Configure a Timer

This will automate the invocation of the flow each day.

In [ ]:
from globus_sdk.scopes import TimerScopes, FlowsScopes
from globus_sdk import TimerClient

In [ ]:
flow_scope = specific_flow_client.scopes.user
end_scope = f"{TimerScopes.timer}[{flow_scope}]"

timer_client = TimerClient(app=my_app, app_scopes=end_scope)

In [ ]:
callback_url = slash_join(specific_flow_client.base_url, f"/flows/{flow_id}/run")

Remove the Y/M/D fields so the flow runs for the current date.

In [ ]:
timer_input = {
    'input': {
        'source': 
        {
            'id': '6c54cade-bde5-45c1-bdea-f4bd71dba2cc',
            'path': '/home/share/godata/'
        },
        'destination': {
            'id': '31ce9ba0-176d-45a5-add3-f37d233ba47d',
            'path': '/~/test/'
        },
        'compute_endpoint': 'a93bab84-bc75-43a0-8ab9-ba7a41a1a2d4',
        'wxt_kwargs': {
            'ndays': 1,
            'site': 'NU',
            'hours': 1,
            'odir': '/home/rchard/src/CROCUS/output/'
        },
        'wxt_function': 'd78ce48d-614a-4c77-add7-1c0e25c3472d'
    }
}

In [ ]:
timer = TimerJob(
    callback_url=callback_url,
    callback_body={"body": flow_input, "label": "CROCUS Timer Flow"},
    start=datetime.datetime.utcnow(),
    interval=datetime.timedelta(seconds=300),
    scope=flow_scope,
    name="CROCUS Flow Timer",
)

In [ ]:
response = timer_client.create_job(timer)

In [ ]:
response

In [ ]:
timer_client.get_job(response.get('job_id')).data

## Delete the timer

In [ ]:
timer_client.delete_job(response.get('job_id'))